### 주제: CarDecko 사이트에 올라온 중고차 가격을 회귀 예측
- CarDecko는 인도의 중고차 관련 웹 사이트

---

#### 사용 데이터 세트 (총 3개)
1. cars_data_clean.csv: 전처리 및 머신 러닝 대상. 총 66개의 컬럼 보유
2. cars_detail_merge.csv: 거의 모든 자동차의 상세 정보(자동차나 소유주의 특징 등)를 가진 데이터 세트
3. feature_dictionary.csv: 위 두 데이터 세트에 포함된 각 컬럼의 정보가 담긴 데이터 세트

In [1]:
import pandas as pd
import numpy as np

# 데이터 세트 불러오기
c_df = pd.read_csv('../datasets/cars_data_clean.csv', low_memory=False)
c_df

,usedCarSkuId,loc,myear,body,transmission,fuel,km,ip,images,imgCount,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
0,7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,69162.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,58.16,6200.0,77.0,3500.0,69.0,NaN
1,c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,45864.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Gasoline Port Injection,11.0,NaN,NaN,58.20,6000.0,78.0,3500.0,73.0,NaN
2,7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,81506.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,86.70,6000.0,109.0,4500.0,NaN,NaN
3,278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,115893.0,0,[{'img': ''}],0,...,Multi-Point Fuel Injection,NaN,13.0,NaN,58.20,6200.0,77.0,3500.0,NaN,NaN
4,b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,18900.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,...,NaN,12.0,NaN,NaN,86.63,5500.0,121.5,4200.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37808,ee6ce523-2aa6-4852-96a1-86b9bc277377,thathawade,2018,suv,manual,diesel,49882.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Common Rail Injection,NaN,16.0,209.0,108.50,3750.0,260.0,2125.0,76.0,82.5
37809,ba1994ac-53ab-4baf-913e-b72f01a37b5d,kharadi,2011,sedan,manual,diesel,82843.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,27,...,Common Rail Injection,NaN,15.0,NaN,88.80,4000.0,200.0,2375.0,NaN,NaN
37810,8f4448ae-53f5-4a3f-b13d-2401dd60c624,kothrud,2020,hatchback,manual,diesel,58000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,11,...,NaN,NaN,16.0,NaN,88.77,4000.0,200.0,2125.0,NaN,NaN
37811,dbf9b629-7215-494b-a62d-ba1408531362,kothrud,2014,hatchback,manual,diesel,62000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,8,...,Common Rail Injection,NaN,NaN,NaN,69.00,4000.0,140.0,2400.0,NaN,NaN


In [2]:
c_df.discountValue.unique()

array([     0,  24000,   6000,   3000,  30000,  94000,   5000,  65000,
       120000,   7000,  10000, 100000,  36000,   4000,  11000, 200000,
       160000,  57000,  37000,  32000,  21000,  50000,  60000, 130000,
       140000,  14000, 165000,  55000,      1,  15000], dtype=int64)

In [3]:
c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37813 entries, 0 to 37812
Data columns (total 66 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   usedCarSkuId              37813 non-null  object 
 1   loc                       31963 non-null  object 
 2   myear                     37813 non-null  int64  
 3   body                      37794 non-null  object 
 4   transmission              37813 non-null  object 
 5   fuel                      37813 non-null  object 
 6   km                        37813 non-null  float64
 7   ip                        37813 non-null  int64  
 8   images                    37813 non-null  object 
 9   imgCount                  37813 non-null  int64  
 10  threesixty                37813 non-null  bool   
 11  dvn                       37813 non-null  object 
 12  oem                       37813 non-null  object 
 13  model                     37813 non-null  object 
 14  varian

In [4]:
import pandas as pd
import numpy as np

# 데이터 세트 불러오기
cd_df = pd.read_csv('../datasets/cars_details_merges.csv', low_memory=False)
cd_df

,position,loc,myear,bt,tt,ft,km,ip,pi,images,...,owner_type,price_segment_new,template_name_new,page_template,template_Type_new,experiment,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
0,1,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
1,2,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
2,3,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
3,4,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,NaN,[{'img': ''}],...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,MPFI,NaN,13,NaN
4,5,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,10+lakh,used cardetail v2,Used Car > Detail Page,used,control,NaN,12.0+-.03,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37809,8,Thathawade,2018,SUV,Manual,Diesel,"49,882",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,second,8lakh-10lakh,used cardetail v2,Used Car > Detail Page,used,control,CDI,NaN,16,209 mm
37810,6,Kharadi,2011,Sedan,Manual,Diesel,"82,843",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2,Used Car > Detail Page,used,control,CRDI,NaN,15,NaN
37811,10,kothrud,2020,Hatchback,Manual,Diesel,"58,000",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,5lakh-8lakh,used cardetail v2,Used Car > Detail Page,used,control,NaN,NaN,16,NaN
37812,20,kothrud,2014,Hatchback,Manual,Diesel,"62,000",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2,Used Car > Detail Page,used,control,CRDi,NaN,NaN,NaN


#### 현재 데이터 세트에 필요한 것들
- cd_df에 정보가 있는 차들만 우선 남길 것
- 결측치 조정: 데이터가 고작 513개인 컬럼이 있기 때문에 제거는 생각해볼 것. 참고로 중복 데이터는 없음
- 차원 축소: **프로젝트의 목적**. 자세한 방식은 로지스틱 회귀 프로젝트 참고할 것
- 인코딩: 문자열 컬럼은 unique()로 클래스 종류 확인해서 인코딩 방식 정할 것
- 표준화 후 이상치 제거: 꿀 가격 예측 프로젝트 참고. 원래 수치형인 컬럼들만 따로 뺄 것

### 데이터 전처리

In [5]:
# 데이터 세트 복사
pre_c_df = c_df.copy()
pre_c_df

,usedCarSkuId,loc,myear,body,transmission,fuel,km,ip,images,imgCount,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
0,7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,69162.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,58.16,6200.0,77.0,3500.0,69.0,NaN
1,c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,45864.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Gasoline Port Injection,11.0,NaN,NaN,58.20,6000.0,78.0,3500.0,73.0,NaN
2,7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,81506.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,86.70,6000.0,109.0,4500.0,NaN,NaN
3,278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,115893.0,0,[{'img': ''}],0,...,Multi-Point Fuel Injection,NaN,13.0,NaN,58.20,6200.0,77.0,3500.0,NaN,NaN
4,b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,18900.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,...,NaN,12.0,NaN,NaN,86.63,5500.0,121.5,4200.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37808,ee6ce523-2aa6-4852-96a1-86b9bc277377,thathawade,2018,suv,manual,diesel,49882.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Common Rail Injection,NaN,16.0,209.0,108.50,3750.0,260.0,2125.0,76.0,82.5
37809,ba1994ac-53ab-4baf-913e-b72f01a37b5d,kharadi,2011,sedan,manual,diesel,82843.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,27,...,Common Rail Injection,NaN,15.0,NaN,88.80,4000.0,200.0,2375.0,NaN,NaN
37810,8f4448ae-53f5-4a3f-b13d-2401dd60c624,kothrud,2020,hatchback,manual,diesel,58000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,11,...,NaN,NaN,16.0,NaN,88.77,4000.0,200.0,2125.0,NaN,NaN
37811,dbf9b629-7215-494b-a62d-ba1408531362,kothrud,2014,hatchback,manual,diesel,62000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,8,...,Common Rail Injection,NaN,NaN,NaN,69.00,4000.0,140.0,2400.0,NaN,NaN


In [7]:
# cd_df(상세 정보 데이터 세트)의 loc 컬럼 정보 출력
cd_df.loc[:, 'loc'].values

array(['Gomti Nagar', 'Borivali West', 'JASOLA', ..., 'kothrud',
       'kothrud', 'ambegaon narhe'], dtype=object)

In [8]:
# pre_c_df에 존재하는 loc 값들 중 cd_df에도 포함된 행들만 유지한 다음 인덱스 초기화 
pre_c_df = pre_c_df[pre_c_df['loc'].isin(cd_df.loc[:, 'loc'].values)].reset_index(drop=True)
pre_c_df

,usedCarSkuId,loc,myear,body,transmission,fuel,km,ip,images,imgCount,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
0,c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,45864.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Gasoline Port Injection,11.0,NaN,NaN,58.20,6000.0,78.0,3500.0,73.0,NaN
1,7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,81506.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,86.70,6000.0,109.0,4500.0,NaN,NaN
2,278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,115893.0,0,[{'img': ''}],0,...,Multi-Point Fuel Injection,NaN,13.0,NaN,58.20,6200.0,77.0,3500.0,NaN,NaN
3,b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,18900.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,...,NaN,12.0,NaN,NaN,86.63,5500.0,121.5,4200.0,NaN,NaN
4,e030b70c-56b9-44b7-8b40-919fa92b6bbc,bengali market,2012,hatchback,manual,cng,60000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,17,...,NaN,NaN,NaN,NaN,58.16,6200.0,77.0,3500.0,69.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37481,ee6ce523-2aa6-4852-96a1-86b9bc277377,thathawade,2018,suv,manual,diesel,49882.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Common Rail Injection,NaN,16.0,209.0,108.50,3750.0,260.0,2125.0,76.0,82.5
37482,ba1994ac-53ab-4baf-913e-b72f01a37b5d,kharadi,2011,sedan,manual,diesel,82843.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,27,...,Common Rail Injection,NaN,15.0,NaN,88.80,4000.0,200.0,2375.0,NaN,NaN
37483,8f4448ae-53f5-4a3f-b13d-2401dd60c624,kothrud,2020,hatchback,manual,diesel,58000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,11,...,NaN,NaN,16.0,NaN,88.77,4000.0,200.0,2125.0,NaN,NaN
37484,dbf9b629-7215-494b-a62d-ba1408531362,kothrud,2014,hatchback,manual,diesel,62000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,8,...,Common Rail Injection,NaN,NaN,NaN,69.00,4000.0,140.0,2400.0,NaN,NaN


#### 중간 분석
- 위 과정으로 약 300개의 데이터 손실 발생

In [9]:
pre_c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37486 entries, 0 to 37485
Data columns (total 66 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   usedCarSkuId              37486 non-null  object 
 1   loc                       31636 non-null  object 
 2   myear                     37486 non-null  int64  
 3   body                      37468 non-null  object 
 4   transmission              37486 non-null  object 
 5   fuel                      37486 non-null  object 
 6   km                        37486 non-null  float64
 7   ip                        37486 non-null  int64  
 8   images                    37486 non-null  object 
 9   imgCount                  37486 non-null  int64  
 10  threesixty                37486 non-null  bool   
 11  dvn                       37486 non-null  object 
 12  oem                       37486 non-null  object 
 13  model                     37486 non-null  object 
 14  varian